# Detecting Malaria cells using Convolutional Neural Network

**Importing Modules**

In [ ]:
import os 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.image import imread

import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing import image

from sklearn.metrics import classification_report, confusion_matrix

**Reading Data**

In [ ]:
data_dir = '../input/malaria-cell-images-dataset/cell_images'

In [ ]:
print(os.listdir(data_dir))

In [ ]:
test_path = data_dir + "/test/"
train_path = data_dir + "/train/"

In [ ]:
test_path

In [ ]:
train_path

In [ ]:
print(os.listdir(test_path))

In [ ]:
print(os.listdir(train_path))

In [ ]:
# os.listdir(test_path + "/parasitized/")

In [ ]:
len(os.listdir(test_path + "/parasitized/")) 

In [ ]:
# os.listdir(test_path + "/uninfected/")

In [ ]:
len(os.listdir(test_path + "/uninfected/")) 

In [ ]:
# os.listdir(train_path + "/parasitized/")

In [ ]:
len(os.listdir(train_path + "/parasitized/"))   

In [ ]:
# os.listdir(train_path + "/uninfected/")

In [ ]:
len(os.listdir(train_path + "/uninfected/"))  

In [ ]:
os.listdir(train_path + "/parasitized/")[0]

In [ ]:
os.listdir(test_path + "/parasitized/")[0]

In [ ]:
para_cell = test_path + "parasitized/" + "C68P29N_ThinF_IMG_20150819_134712_cell_55.png"

In [ ]:
para_cell

In [ ]:
# imread(para_cell)

In [ ]:
imread(para_cell).max()

In [ ]:
imread(para_cell).min()

In [ ]:
imread(para_cell).shape

In [ ]:
plt.imshow(imread(para_cell))

In [ ]:
os.listdir(test_path + "/uninfected/")[0]

In [ ]:
uninfected_cell = test_path + "uninfected/" + "C181P142ThinF_IMG_20151127_155722_cell_78.png"

In [ ]:
plt.imshow(imread(uninfected_cell))

In [ ]:
dim1 = []
dim2 = []

for img_filename in os.listdir(test_path + "uninfected"):
    img = imread(test_path + "uninfected/" + img_filename)
    d1, d2, colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
# dim1

In [ ]:
# dim2

In [ ]:
np.mean(dim1)

In [ ]:
np.mean(dim2)

In [ ]:
image_shape = (130, 130, 3)   # 130 * 130 * 3

In [ ]:
sns.jointplot(x=dim1, y=dim2)

**Data Processing**

In [ ]:
# help(ImageDataGenerator)

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              horizontal_flip=True,
                              fill_mode='nearest')

In [ ]:
para_img = imread(para_cell)
plt.imshow(para_img)

In [ ]:
plt.imshow(image_gen.random_transform(para_img))

In [ ]:
image_gen.flow_from_directory(train_path)

In [ ]:
image_gen.flow_from_directory(test_path)

**Create the Mode**

In [ ]:
image_shape = (130, 130, 3)

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 16

In [ ]:
image_shape[:2]

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',
                                               shuffle=False)

In [ ]:
train_image_gen.class_indices

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
results = model.fit(train_image_gen, epochs=20,
                    validation_data=test_image_gen,
                    callbacks=[early_stop])

In [ ]:
model.save('malaria_detector.h5')

In [ ]:
model.history.history

**Evaluating the Model**

In [ ]:
losses = pd.DataFrame(model.history.history)
losses[['loss', 'val_loss']].plot()

In [ ]:
model.evaluate(test_image_gen)

In [ ]:
model.metrics_names

In [ ]:
pred = model.predict(test_image_gen)

In [ ]:
pred

In [ ]:
predictions = pred > 0.5

In [ ]:
predictions

In [ ]:
len(pred)

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
test_image_gen.classes

In [ ]:
print(classification_report(test_image_gen.classes, predictions))

In [ ]:
print(confusion_matrix(test_image_gen.classes, predictions))

In [ ]:
# How to predict on a single image 

In [ ]:
para_cell

In [ ]:
image.load_img(para_cell)

In [ ]:
type(image.load_img(para_cell))

In [ ]:
my_image = image.load_img(para_cell, target_size=image_shape)

In [ ]:
my_image

In [ ]:
my_image_arr = image.img_to_array(my_image)

In [ ]:
my_image

In [ ]:
# my_image_arr

In [ ]:
my_image_arr.shape   # one image (1, 130, 130, 3)

In [ ]:
my_image_arr = np.expand_dims(my_image_arr, axis=0)

In [ ]:
my_image_arr.shape

In [ ]:
model.predict(my_image_arr)   # parasitized

In [ ]:
test_image_gen.class_indices

In [1]:
import os

os.getcwd()

'/home/kashi/Documents/USMAN BHAI/Tranfer learning'